<a href="https://colab.research.google.com/github/lohan20s/Predict_Healthcare_costs_linearregression/blob/main/fcc_predict_health_costs_with_regression_SL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
#check if there are any invalid numbers
dataset.isna().sum()


In [ ]:
#Convert categorical columns data in the above dataset to numbers (like 0,1)
dataset_cat = dataset.copy()
dataset_cat['sex'] = dataset_cat['sex'].map({'male': 1, 'female': 0})
dataset_cat['smoker'] = dataset_cat['smoker'].map({'yes': 1, 'no': 0})
dataset_cat['region'] = dataset_cat['region'].map({'northwest': 0, 'northeast': 1, 'southeast': 2, 'southwest': 3})

print(dataset_cat.head())


In [ ]:
#Use 80% of the data as the train_dataset and 20% of the data as the test_dataset.
train_dataset = dataset_cat.sample(frac=0.8, random_state=0)
test_dataset = dataset_cat.drop(train_dataset.index)
print(train_dataset.head())
print(test_dataset.head())

In [ ]:
#remove the "expenses" column from these datasets to create new datasets and labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')
numfeatures=len(train_dataset.columns)

In [ ]:
# Normalize the input features
normalizer = layers.Normalization()
normalizer.adapt(np.array(train_dataset)) #save the mean and variance from the training dataset

In [ ]:
#Create a keras model and train it with the train_dataset.
model = keras.Sequential([
    normalizer,
    layers.Dense(units=numfeatures, activation='relu'), # add a hidden layer for training stability
    layers.Dense(units=1), # add the linear regression layer
])
model.summary()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error',metrics=['mean_absolute_error', 'mean_squared_error'])


In [ ]:
#train the model with the training dataset
history=model.fit(train_dataset,train_labels, epochs=150, verbose=0)
model.summary()

#plot mae over epochs to determine the correct number of epochs
mae_history = history.history['mean_absolute_error']

plt.plot(range(1, len(mae_history) + 1), mae_history, label='MAE')
plt.title('Mean Absolute Error Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


In [ ]:
#This is not part of the project, but we can also develop a linear model using statsmodels (this will be a closed form solution)
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error

# Add intercept and fit the model
X_const = sm.add_constant(train_dataset)
model = sm.OLS(train_labels, X_const).fit()

# print model coefficients
print("Statsmodels Coefficients:\n", model.params)

# compute mae for the test dataset
y_pred = model.predict(sm.add_constant(test_dataset))
mae = mean_absolute_error(test_labels, y_pred)
print(f"Mean Absolute Error on test dataset: {mae:.4f}")

In [ ]:
#This is not part of the project, but we can also develop a linear model using sklearn (this will be a closed form solution)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Create and fit model
model = LinearRegression(fit_intercept=True)
model.fit(train_dataset, train_labels)

# print model coefficients
print("\nSklearn Coefficients:\n", np.append(model.intercept_, model.coef_))

#compute mae
y_pred = model.predict(test_dataset)
mae = mean_absolute_error(test_labels, y_pred)
print(f"Mean Absolute Error on test dataset: {mae:.4f}")